In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate=0.001
training_epochs=30
batch_size=100

In [14]:
fashionmnist_train=dsets.FashionMNIST(root='data/MNIST_data',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
fashionmnist_test=dsets.FashionMNIST(root='data/MNIST_data',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [5]:
data_loader=DataLoader(dataset=fashionmnist_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2=nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc=nn.Linear(7*7*64, 10, bias=True) # 완전연결층
        nn.init.xavier_uniform_(self.fc.weight) # 완전연결층 한정 가중치 초기화

    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out

In [7]:
model=CNN().to(device)
criterion=nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
total_batch=len(data_loader)
print(total_batch)

600


In [11]:
for epoch in range(training_epochs):
    avg_cost=0
    avg_accuracy=0

    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat=model(X)
        cost=criterion(y_hat, Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch
    print('epoch: ', epoch, '\n', 'avg_cost: ', avg_cost, '\n', 'cost: ', cost.item())

epoch:  0 
 avg_cost:  tensor(0.3356, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.257686048746109
epoch:  1 
 avg_cost:  tensor(0.2890, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.26596641540527344
epoch:  2 
 avg_cost:  tensor(0.2624, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.19059023261070251
epoch:  3 
 avg_cost:  tensor(0.2420, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.21950402855873108
epoch:  4 
 avg_cost:  tensor(0.2261, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.21191073954105377
epoch:  5 
 avg_cost:  tensor(0.2115, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.17408986389636993
epoch:  6 
 avg_cost:  tensor(0.2000, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.14954382181167603
epoch:  7 
 avg_cost:  tensor(0.1879, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.14955396950244904
epoch:  8 
 avg_cost:  tensor(0.1773, device='cuda:0', grad_fn=<AddBackward0>) 
 cost:  0.11129867285490036
epoch:  9 
 avg_cost:  tensor(

In [15]:
with torch.no_grad():
  X_test = fashionmnist_test.test_data.view(len(fashionmnist_test), 1, 28, 28).float().to(device)
  Y_test = fashionmnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print(accuracy.item())

0.774399995803833
